<a href="https://colab.research.google.com/github/lunecarvalho/similarity-sentences-nlp/blob/main/similarity_sentences_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
url = 'https://raw.githubusercontent.com/alura-cursos/NLP-trabalhando-similaridade-sentencas/refs/heads/main/reviews_zoop.csv'

In [ ]:
import pandas as pd

dados = pd.read_csv(url)
dados

In [ ]:
dados['review'][1]

In [ ]:
dados['review'][8]

In [ ]:
dados['review'][11]

In [ ]:
dados['review'][18]

In [ ]:
from nltk.tokenize import word_tokenize
from functools import partial

word_tokenize_pt = partial(word_tokenize, language='portuguese')

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
!pip install gensim

In [ ]:
from gensim.models import Word2Vec

modelo = Word2Vec(sentences = dados['review'].apply(word_tokenize_pt),
                  vector_size=100, min_count=1, window=5,
                  workers=1, seed=45)

In [ ]:
modelo.wv.most_similar('entrega')

In [ ]:
modelo.wv.most_similar('vendedor')

In [ ]:
modelo.wv.most_similar('produto')

In [ ]:
dados['review_token'] = dados['review'].apply(word_tokenize_pt)

In [ ]:
from gensim.models.doc2vec import TaggedDocument

dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)])
             for i, linha in dados.iterrows()]

In [ ]:
dados_tag

In [ ]:
from gensim.models import Doc2Vec

modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers=1, seed=45, epochs=20)

In [ ]:
vetor_inferido = modelo.infer_vector(['entrega'])
vetor_inferido

In [ ]:
frases_similares = modelo.dv.most_similar([vetor_inferido])
frases_similares

In [ ]:
for idx, similaridade in frases_similares:
  print(f'Review: {dados.iloc[int(idx)]["review"]} - Similaridade: {similaridade:.4f}')

In [ ]:
nltk.download('stopwords')

In [ ]:
import re
from nltk.corpus import stopwords

def tratamento_inicial(texto):
  texto = re.sub(r'\W', ' ', texto.lower())

  tokens = word_tokenize_pt(texto)
  stop_words = set(stopwords.words('portuguese'))
  stop_words.discard('não')

  return ' '.join([w for w in tokens if w not in stop_words])

In [ ]:
dados['tratamento_1'] = dados['review'].apply(tratamento_inicial)

In [ ]:
dados['tratamento_1']

In [ ]:
!pip install unidecode

In [ ]:
import unidecode

sem_acentos = [unidecode.unidecode(texto) for texto in dados['tratamento_1']]

In [ ]:
dados['tratamento_2'] = sem_acentos
dados['tratamento_2']

In [ ]:
dados['review_token'] = dados['tratamento_2'].apply(word_tokenize_pt)
dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)])
             for i, linha in dados.iterrows()]
modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers=1, seed=45, epochs=20)
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])], topn=5)
for idx, similaridade in frases_similares:
  print(f'Review: {dados.iloc[int(idx)]["tratamento_2"]} - Similaridade: {similaridade:.4f}')

In [ ]:
df = dados.drop_duplicates(subset='tratamento_2')
df

In [ ]:
df['tratamento_2'][3319]

In [ ]:
df = df[df['tratamento_2'] != '']

In [ ]:
def normalizar_repeticoes(texto):
  return re.sub(r'(?!rr|ss)(.)\1+', r'\1', texto)

df['tratamento_3'] = df['tratamento_2'].apply(normalizar_repeticoes)

In [ ]:
df['tratamento_3'][3319]

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
!pip install stanza
!python -m stanza.download pt

In [ ]:
import stanza

stanza.download('pt')
nlp = stanza.Pipeline('pt',
                      processors='tokenize,lemma',
                      use_gpu=False,
                      batch_size=64,
                      n_process=4)

In [ ]:
def lematizar_texto(textos):
  texto_lematizado = []

  for texto in textos:
    doc_frase = nlp(texto)
    frase_lematizada = ' '.join([palavra.lemma for frase in doc_frase.sentences for palavra in frase.words])
    texto_lematizado.append(frase_lematizada)

  return texto_lematizado

In [ ]:
textos = ['gostei muito experiencia comprar',
          'minha filha gostou produto',
          'compra foi facil compra rapida']
lematizar_texto(textos)

In [ ]:
df['tratamento_4'] = lematizar_texto(df['tratamento_3'])
df['tratamento_4']

In [ ]:
df_lem = df.drop_duplicates(subset='tratamento_4')
df_lem

In [ ]:
df_lem.reset_index(drop=True, inplace=True)

In [ ]:
df_lem['review_token'] = df_lem['tratamento_4'].apply(word_tokenize_pt)
dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)])
             for i, linha in df_lem.iterrows()]
modelo = Doc2Vec(dados_tag, vector_size=100, min_count=2, window=2,
                 workers=1, seed=45, epochs=20)
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])], topn=5)
for idx, similaridade in frases_similares:
  print(f'Review: {df_lem.iloc[int(idx)]["tratamento_4"]} - Similaridade: {similaridade:.4f}')

In [ ]:
modelo = Doc2Vec(vector_size=300, min_count=2,
                 window=5, workers=1, seed=45, epochs=20)
modelo.build_vocab(dados_tag)

modelo.train(dados_tag, total_examples=modelo.corpus_count,
             epochs=modelo.epochs)

In [ ]:
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])], topn=5)

for idx, similaridade in frases_similares:
  print(f'Review: {df_lem.iloc[int(idx)]["tratamento_1"]} - Similaridade: {similaridade:.4f}')

In [ ]:
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['vendedor'])], topn=5)

for idx, similaridade in frases_similares:
  print(f'Review: {df_lem.iloc[int(idx)]["tratamento_1"]} - Similaridade: {similaridade:.4f}')

In [ ]:
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['reputacao loja'])], topn=5)

for idx, similaridade in frases_similares:
  print(f'Review: {df_lem.iloc[int(idx)]["tratamento_1"]} - Similaridade: {similaridade:.4f}')

In [ ]:
modulo_url ='https://tfhub.dev/google/universal-sentence-encoder/4'

In [ ]:
import tensorflow_hub as hub

modelo = hub.load(modulo_url)

In [ ]:
modelo

In [ ]:
reviews = df['tratamento_3'].tolist()
reviews_emb = modelo(reviews)
reviews_emb

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(reviews_emb)

In [ ]:
import numpy as np

def sentencas_similares(tema, reviews, reviews_emb, top_n=5):
  tema_emb = modelo([tema])
  similaridades = cosine_similarity(tema_emb, reviews_emb).flatten()

  indices_similares = np.argsort(-similaridades)
  for idx in indices_similares[:top_n]:
    print(f'Review: {reviews[idx]} - Similaridade: {similaridades[idx]:.4f}')

In [ ]:
sentencas_similares('entrega', reviews, reviews_emb)

In [ ]:
sentencas_similares('reputação loja', reviews, reviews_emb, top_n=10)

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

modulo_url ='https://tfhub.dev/google/universal-sentence-encoder/4'
modelo = hub.load(modulo_url)

def tratar_texto(texto):
  texto = re.sub(r'\W', ' ', texto.lower())

  word_tokenize_pt = partial(word_tokenize, language='portuguese')
  tokens = word_tokenize_pt(texto)
  stop_words = set(stopwords.words('portuguese'))
  stop_words.discard('não')
  tokens = [w for w in tokens if w not in stop_words]

  texto_sem_acentos = unidecode.unidecode(' '.join(tokens))

  texto_normalizado = re.sub(r'(?!rr|ss)(.)\1+', r'\1', texto_sem_acentos)

  return texto_normalizado

def process_csv(arquivo, tema):
  df = pd.read_csv(arquivo.name)

  df['review_tratada'] = df['review'].apply(tratar_texto)
  df = df[df['review_tratada']!='']
  df.drop_duplicates(subset='review_tratada', inplace=True)

  reviews_emb = modelo(df['review_tratada'].tolist())
  tema_emb = modelo([tema])

  similaridades = cosine_similarity(tema_emb, reviews_emb).flatten()
  top_indices = np.argsort(-similaridades)

  similar_reviews = df[['nota_review', 'review']].iloc[top_indices]
  similar_df = similar_reviews.head(200)
  nome_arquivo = f'reviews_similares_{tema}.csv'
  similar_reviews.to_csv(nome_arquivo)

  return similar_df, nome_arquivo

with gr.Blocks() as app:
  with gr.Row():
    gr.Markdown('## Encontrando as reviews mais similares ao tema')
  csv_entrada = gr.File(label='Envie o CVS com as reviews', file_types=['.csv'])
  tema_entrada = gr.Textbox(label='Digite o tema de busca (Ex.: "Entrega")')

  botao = gr.Button('Clique para buscas as reviews')

  tabela_saida = gr.Dataframe(label='Top 200 reviews similares', headers=['Nota', 'Reviews'], interactive=False)
  arquivo_saida = gr.File(label='Baixar CSV ordenado com as reviews mais similares ao tema', interactive=False)

  botao.click(process_csv, inputs=[csv_entrada, tema_entrada], outputs=[tabela_saida, arquivo_saida])
app.launch()

In [ ]:
process_csv(url, 'entrega')